In [1]:
import xarray as xr
import numpy as np
from dask import array as darr
from dask.distributed import Client

In [2]:
client = Client('tcp://127.0.0.1:43049')
client

<Client: 'tcp://127.0.0.1:43049' processes=4 threads=16, memory=64.00 GiB>

In [9]:
# parameters
num_nodes = 22_000
chunks_nodes = 2_000
num_neighbors = 10
len_time = 4_000
chunks_time = 40
len_z = 40
chunks_z = 10

In [10]:
n = xr.DataArray(
    np.arange(num_nodes, ),
    dims=("n", ),
    name="n",
)
n

<xarray.DataArray 'n' (n: 22000)>
array([    0,     1,     2, ..., 21997, 21998, 21999])
Dimensions without coordinates: n

In [11]:
neighbors = xr.DataArray(
    np.arange(num_neighbors),
    dims=("j", ),
    name="neighbor",
)

In [12]:
z = xr.DataArray(
    np.arange(len_z),
    dims=("z", ),
    name="z",
)
z

<xarray.DataArray 'z' (z: 40)>
array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39])
Dimensions without coordinates: z

In [13]:
groups = (
    n.isel(n=slice(None, None, num_neighbors))
    + neighbors
)
groups

<xarray.DataArray (n: 2200, j: 10)>
array([[    0,     1,     2, ...,     7,     8,     9],
       [   10,    11,    12, ...,    17,    18,    19],
       [   20,    21,    22, ...,    27,    28,    29],
       ...,
       [21970, 21971, 21972, ..., 21977, 21978, 21979],
       [21980, 21981, 21982, ..., 21987, 21988, 21989],
       [21990, 21991, 21992, ..., 21997, 21998, 21999]])
Dimensions without coordinates: n, j

In [14]:
T = xr.DataArray(
    darr.random.normal(
        size=(num_nodes, len_time, len_z),
        chunks=(chunks_nodes, chunks_time, chunks_z),
    ),
    dims=("n", "t", "z"),
    name="temp",
    coords={"n": n, "z": z},
)
T

<xarray.DataArray 'temp' (n: 22000, t: 4000, z: 40)>
dask.array<normal, shape=(22000, 4000, 40), dtype=float64, chunksize=(2000, 40, 10), chunktype=numpy.ndarray>
Coordinates:
  * n        (n) int64 0 1 2 3 4 5 6 ... 21994 21995 21996 21997 21998 21999
  * z        (z) int64 0 1 2 3 4 5 6 7 8 9 10 ... 30 31 32 33 34 35 36 37 38 39
Dimensions without coordinates: t

In [15]:
W = xr.DataArray(
    darr.ones(
        shape=(num_nodes, num_neighbors),
        chunks=(chunks_nodes, num_neighbors),
    ),
    dims=("n", "j"),
    name="weights",
)
W = W.assign_coords(n=n)
W

<xarray.DataArray 'weights' (n: 22000, j: 10)>
dask.array<ones_like, shape=(22000, 10), dtype=float64, chunksize=(2000, 10), chunktype=numpy.ndarray>
Coordinates:
  * n        (n) int64 0 1 2 3 4 5 6 ... 21994 21995 21996 21997 21998 21999
Dimensions without coordinates: j

In [16]:
T_prime = (T * W).sel(n=groups).sum("j") / W.sel(n=groups).sum("j")
T_prime

<xarray.DataArray (n: 2200, t: 4000, z: 40)>
dask.array<truediv, shape=(2200, 4000, 40), dtype=float64, chunksize=(2200, 40, 10), chunktype=numpy.ndarray>
Coordinates:
  * z        (z) int64 0 1 2 3 4 5 6 7 8 9 10 ... 30 31 32 33 34 35 36 37 38 39
Dimensions without coordinates: n, t

In [17]:
T_prime = T_prime.compute()
T_prime

/srv/conda/envs/notebook/lib/python3.11/site-packages/distributed/client.py:3163: UserWarning: Sending large graph of size 78.17 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


<xarray.DataArray (n: 2200, t: 4000, z: 40)>
array([[[ 0.35359622,  0.58177528, -0.32958708, ..., -0.51837728,
         -0.15589975,  0.08751039],
        [ 0.07256969,  0.47753047,  0.47401105, ...,  0.22361446,
         -0.16893054, -0.34140336],
        [ 0.17216508,  0.31142422,  0.1073315 , ..., -0.3373018 ,
          0.64867212, -0.21370211],
        ...,
        [ 0.37977104,  0.02624474,  0.82345729, ...,  0.03463916,
         -0.10669639,  0.71840686],
        [ 0.14493048, -0.50145331, -0.35249146, ...,  0.24791022,
          0.12752712,  0.04626096],
        [-0.03839463,  0.16633031,  0.32488272, ...,  0.11599471,
          0.04750003, -0.04680481]],

       [[-0.09937241, -0.64161511, -0.07936227, ..., -0.19943057,
         -0.48909161,  0.17974529],
        [-0.39914211, -0.14694799,  0.45453064, ..., -0.57208561,
          0.43073535,  0.02933821],
        [ 0.44454389,  0.34243203, -0.03329581, ..., -0.07671047,
          0.37410905,  0.24221656],
...
        [ 0.02468047,  0.02471272,  0.0909833 , ...,  0.41468093,
         -0.28219032, -0.22634626],
        [-0.29144649, -0.19891412,  0.07291238, ...,  0.08569272,
          0.39062838,  0.16963788],
        [ 0.45542376, -0.13306641,  0.26449713, ...,  0.12172884,
         -0.06523543,  0.2001059 ]],

       [[-0.51244448, -0.10407603,  0.53850115, ...,  0.76387538,
          0.10919007, -0.27977432],
        [-0.42459243, -0.09636989,  0.57137489, ...,  0.54267271,
         -0.04461874, -0.48750497],
        [-0.10368388,  0.21056813,  0.15675724, ...,  0.25313467,
          0.0912293 , -0.49394391],
        ...,
        [ 0.23799814, -0.24661726, -0.23093793, ..., -0.16994797,
          0.17314961,  0.40437193],
        [ 0.3461073 , -0.41903546,  0.02239471, ..., -0.36243337,
          0.11592984,  0.33902717],
        [ 0.08266093,  0.14058535, -0.37805522, ..., -0.25460486,
         -0.08747237, -0.11376788]]])
Coordinates:
  * z        (z) int64 0 1 2 3 4 5 6 7 8 9 10 ... 30 31 32 33 34 35 36 37 38 39
Dimensions without coordinates: n, t